https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/overview

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

/opt/conda/lib/python3.7/site-packages/fastbook/__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
from fastbook import * 

In [3]:
import pandas as pd
import re

In [119]:
from tqdm import tqdm

In [144]:
import tqdm.notebook as tq

In [146]:
from tqdm.notebook import tqdm

In [4]:
sample_sub = pd.read_csv("data/sample_submission_stg2.csv"); sample_sub

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
1,img_00007.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
2,img_00009.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
3,img_00018.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
4,img_00027.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
...,...,...,...,...,...,...,...,...,...
13148,test_stg2/image_12149.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13149,test_stg2/image_12150.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13150,test_stg2/image_12151.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283
13151,test_stg2/image_12152.jpg,0.455003,0.052938,0.030969,0.017734,0.123081,0.079142,0.046585,0.194283


In [5]:
sample_sub.columns

Index(['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')

# Load Data and Model

In [163]:
def get_dls(bs, item_size, aug_size):
    return ImageDataLoaders.from_folder(
        path = "data/train",
        valid_pct = 0.2,
        seed = 1991,
        item_tfms = Resize(item_size),
        batch_tfms = aug_transforms(size = aug_size),
        bs = bs
    )

In [164]:
dls = get_dls(64, 448, 224)

In [165]:
dls.vocab

(#8) ['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT']

In [166]:
learn = cnn_learner(dls, resnet34, metrics = accuracy)

In [167]:
model = "resnet34_stage2_aug12_ls"

In [168]:
learn.load(model)

# Create Submission File

In [170]:
test_items1 = get_image_files(Path("data/test_stg1"))
test_items2 = get_image_files(Path("data/test_stg2"))

In [171]:
test_dl1 = dls.test_dl(test_items1)
test_dl2 = dls.test_dl(test_items2)

In [172]:
preds1,_ = learn.get_preds(dl=test_dl1)
preds2,_ = learn.get_preds(dl=test_dl2)

In [190]:
def modify_preds(preds, min_value, max_value):
    filt1 = preds > max_value
    filt2 = preds < min_value
    preds[filt1] = max_value
    preds[filt2] = min_value
    return preds

In [192]:
preds1 = modify_preds(preds1, 0.01, 0.95)
preds2 = modify_preds(preds2, 0.01, 0.95)

In [193]:
columns = sample_sub.columns
# columns = columns.drop(["image", 'ALB', 'BET', 'YFT']); columns
columns = columns.drop(["image"]); columns

Index(['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')

In [194]:
df1 = pd.DataFrame(data = preds1, columns = columns)
df2 = pd.DataFrame(data = preds2, columns = columns)

In [195]:
def get_file_name(path):
    match = re.match('^.*\/([^\/]+\.jpg)', str(path))
    return match.group(1)

In [196]:
test_items1_names = [get_file_name(path) for path in test_items1]
test_items2_names = ["test_stg2/" + get_file_name(path) for path in test_items2]

In [197]:
df1.insert(loc = 0, column = "image", value = test_items1_names)
df2.insert(loc = 0, column = "image", value = test_items2_names)

In [198]:
submission_df = pd.concat([df1, df2]); submission_df

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00950.jpg,0.010000,0.010000,0.010000,0.010000,0.950000,0.010000,0.010000,0.010000
1,img_01384.jpg,0.308409,0.067140,0.038380,0.024461,0.042242,0.351166,0.035417,0.132785
2,img_03252.jpg,0.945627,0.012550,0.010000,0.010000,0.010000,0.010000,0.010398,0.010000
3,img_00138.jpg,0.650138,0.011643,0.020711,0.019940,0.198678,0.010000,0.023596,0.071578
4,img_04073.jpg,0.010000,0.044687,0.010000,0.010000,0.010000,0.010000,0.010000,0.940568
...,...,...,...,...,...,...,...,...,...
12148,test_stg2/image_06524.jpg,0.460297,0.101537,0.023048,0.011845,0.039846,0.013379,0.015135,0.334911
12149,test_stg2/image_02773.jpg,0.021524,0.038833,0.057649,0.106832,0.335798,0.055370,0.139789,0.244205
12150,test_stg2/image_05407.jpg,0.406232,0.182009,0.077482,0.020413,0.139474,0.024743,0.010981,0.138666
12151,test_stg2/image_00875.jpg,0.748587,0.010000,0.014849,0.010000,0.190661,0.010000,0.010000,0.025082


In [199]:
submission_df = submission_df.sort_values(by="image") ; submission_df

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
664,img_00005.jpg,0.010000,0.010000,0.010000,0.010000,0.950000,0.010000,0.010000,0.010000
760,img_00007.jpg,0.098225,0.385364,0.037294,0.057968,0.041102,0.025343,0.010000,0.345457
415,img_00009.jpg,0.950000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
632,img_00018.jpg,0.950000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.013146
347,img_00027.jpg,0.520176,0.100177,0.016380,0.025615,0.029543,0.131524,0.010000,0.169494
...,...,...,...,...,...,...,...,...,...
8237,test_stg2/image_12149.jpg,0.445995,0.211006,0.021084,0.020461,0.206694,0.025697,0.019446,0.049617
2037,test_stg2/image_12150.jpg,0.493906,0.058389,0.010000,0.010000,0.391896,0.025731,0.010000,0.010370
6635,test_stg2/image_12151.jpg,0.293545,0.076424,0.048611,0.036176,0.087113,0.012721,0.309302,0.136109
6382,test_stg2/image_12152.jpg,0.257974,0.019144,0.023477,0.059719,0.351986,0.182348,0.060635,0.044718


In [200]:
file_path = "data/" + model

In [201]:
submission_df.to_csv(file_path, index = False)

# Submitting to Kaggle

In [202]:
competition =  "the-nature-conservancy-fisheries-monitoring"

In [203]:
!kaggle competitions submit the-nature-conservancy-fisheries-monitoring -f $file_path -m $model 

100%|███████████████████████████████████████| 1.34M/1.34M [00:02<00:00, 634kB/s]
Successfully submitted to The Nature Conservancy Fisheries Monitoring